# Imports

In [1]:
import pandas as pd
import numpy as np
import trueskill

from pandas import Series, DataFrame

from trueskill import Rating, quality_1vs1, rate_1vs1

# Source and Destination Files

In [2]:
# Place source and destination files here
# Imports first
# Write the current season here so you get the correct csvs
season = '2016a'
#season_year = = ''.join(x for x in season if x.isdigit())
#season_year = int(season_year)
#season_ab = season[-1]
results_csv = '2016a_results_ladies.csv'
oldskill_csv = 'skill ratings 2015b.csv'

# Outputs second
winloss_csv = '2016a winloss.csv'
teams_csv = '2016a_teams.csv'
newskill_csv = 'skill ratings 2016a.csv'

In [3]:
results = pd.read_csv(results_csv)
results.head()

,ROUND,HOME,SCORE,SCORE.1,AWAY,OT
0,1,Bayswatermelon Breezers,8.0,14.0,Ardross A-rerollers,NaN
1,1,Chick Park,7.0,9.0,Como Cunstables,NaN
2,1,Dianella Dirty Street Pies,5.0,20.0,Coolbinia Cruisers,NaN
3,1,Freopian tubes,6.0,5.0,Jolimont Flying Vs,NaN
4,1,Freovaries,6.0,9.0,Applecross Dressers,NaN


In [4]:
for i in range(0,len(results)):
    results['OT'][i] = results['OT'][i] == "OT"

results.head()

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,ROUND,HOME,SCORE,SCORE.1,AWAY,OT
0,1,Bayswatermelon Breezers,8.0,14.0,Ardross A-rerollers,0.0
1,1,Chick Park,7.0,9.0,Como Cunstables,0.0
2,1,Dianella Dirty Street Pies,5.0,20.0,Coolbinia Cruisers,0.0
3,1,Freopian tubes,6.0,5.0,Jolimont Flying Vs,0.0
4,1,Freovaries,6.0,9.0,Applecross Dressers,0.0


In [5]:
results['IsDraw'] = results['SCORE'] == results['SCORE.1']

for i in range(0,len(results)):
    if results['OT'][i]:
        results['IsDraw'][i] = True
        
results.head()

,ROUND,HOME,SCORE,SCORE.1,AWAY,OT,IsDraw
0,1,Bayswatermelon Breezers,8.0,14.0,Ardross A-rerollers,0.0,False
1,1,Chick Park,7.0,9.0,Como Cunstables,0.0,False
2,1,Dianella Dirty Street Pies,5.0,20.0,Coolbinia Cruisers,0.0,False
3,1,Freopian tubes,6.0,5.0,Jolimont Flying Vs,0.0,False
4,1,Freovaries,6.0,9.0,Applecross Dressers,0.0,False


In [6]:
# Work out if a game was actually played, and if so who won

results['Winteam']=np.NaN
results['Lossteam']=np.NaN
results['gameplayed']=np.NaN
results = results.rename(columns = {"SCORE":"homescore","SCORE.1":"awayscore"})
for i in range(0,len(results)):
    results['gameplayed'][i] = results['homescore'][i] != 0 and results['awayscore'][i] != 0
    # If home score is greater than away score
    if results['homescore'][i]>=results['awayscore'][i]:
        results['Winteam'][i]=results['HOME'][i]
        results['Lossteam'][i]=results['AWAY'][i]
    else:
        results['Winteam'][i]=results['AWAY'][i]
        results['Lossteam'][i]=results['HOME'][i]

    
results.head()

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda2\lib\site-packages\pandas\core\indexing.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

,ROUND,HOME,homescore,awayscore,AWAY,OT,IsDraw,Winteam,Lossteam,gameplayed
0,1,Bayswatermelon Breezers,8.0,14.0,Ardross A-rerollers,0.0,False,Ardross A-rerollers,Bayswatermelon Breezers,1.0
1,1,Chick Park,7.0,9.0,Como Cunstables,0.0,False,Como Cunstables,Chick Park,1.0
2,1,Dianella Dirty Street Pies,5.0,20.0,Coolbinia Cruisers,0.0,False,Coolbinia Cruisers,Dianella Dirty Street Pies,1.0
3,1,Freopian tubes,6.0,5.0,Jolimont Flying Vs,0.0,False,Freopian tubes,Jolimont Flying Vs,1.0
4,1,Freovaries,6.0,9.0,Applecross Dressers,0.0,False,Applecross Dressers,Freovaries,1.0


In [7]:
results = results.drop(results.columns[[0,1,2,3,4,5]],axis=1)
winloss = results[results["gameplayed"] == 1]

In [8]:
x = Series(winloss['Winteam'].unique())
y = Series(winloss['Lossteam'].unique())
x=x.append(y)
x=Series(x.unique())
x=x.dropna()
x.sort_values()


4                Applecross Dressers
0                Ardross A-rerollers
12           Bayswatermelon Breezers
10              Beachside Belle Ends
9                         Chick Park
1                    Como Cunstables
2                 Coolbinia Cruisers
8         Dianella Dirty Street Pies
11               Dirty Herdy Birdies
7     Fremantle Cappuccino Strippers
15                    Freopian Tubes
3                     Freopian tubes
14                        Freovaries
13                Jolimont Flying Vs
5           Joondnanna Jerryhatricks
6                      Perth On Heat
dtype: object

In [9]:
#winloss.head()


In [10]:
x=Series(x.unique())
x=x.sort_values()
#teams_lower = Series([])
#x = Series(winloss['Winteam'].unique())
#y = Series(winloss['Lossteam'].unique())
#x=x.append(y)
#x=Series(x.unique())

#teams = x.sort_values()
for i in range(0,len(x)):
    x[i]=x[i].lower()
teams = x

In [11]:
teams.unique()

array(['applecross dressers', 'ardross a-rerollers',
       'bayswatermelon breezers', 'beachside belle ends', 'chick park',
       'como cunstables', 'coolbinia cruisers',
       'dianella dirty street pies', 'dirty herdy birdies',
       'fremantle cappuccino strippers', 'freopian tubes', 'freovaries',
       'jolimont flying vs', 'joondnanna jerryhatricks', 'perth on heat'], dtype=object)

In [12]:
# Write to CSV, manually apply lower() to the winteam and lossteam
winloss.to_csv(winloss_csv)
teams.to_csv(teams_csv)

In [13]:
#teams_lower = pd.read_csv(teams_csv)
#teams = teams_lower['team']

# Reread the manually fixed stuff
winloss = pd.read_csv(winloss_csv)

In [14]:
winloss = winloss.dropna()
for i in range(0,len(winloss)):
    winloss['Winteam'][i] = winloss['Winteam'][i].lower()
    winloss['Lossteam'][i] = winloss['Lossteam'][i].lower()
    
winloss.head()

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Unnamed: 0,IsDraw,Winteam,Lossteam,gameplayed
0,0,False,ardross a-rerollers,bayswatermelon breezers,1.0
1,1,False,como cunstables,chick park,1.0
2,2,False,coolbinia cruisers,dianella dirty street pies,1.0
3,3,False,freopian tubes,jolimont flying vs,1.0
4,4,False,applecross dressers,freovaries,1.0


In [15]:
#winteams = Series(winloss['Winteam'].unique())
#lossteams = Series(winloss['Lossteam'].unique())
#teams = winteams
#teams = teams.append(lossteams)
teams = Series(teams.unique())
print(teams);
print len(teams), "teams";

0                applecross dressers
1                ardross a-rerollers
2            bayswatermelon breezers
3               beachside belle ends
4                         chick park
5                    como cunstables
6                 coolbinia cruisers
7         dianella dirty street pies
8                dirty herdy birdies
9     fremantle cappuccino strippers
10                    freopian tubes
11                        freovaries
12                jolimont flying vs
13          joondnanna jerryhatricks
14                     perth on heat
dtype: object
15 teams


# Create skill from lower(teams), update skill

In [16]:
skill = {name:Rating() for name in teams}
skill

{'applecross dressers': trueskill.Rating(mu=25.000, sigma=8.333),
 'ardross a-rerollers': trueskill.Rating(mu=25.000, sigma=8.333),
 'bayswatermelon breezers': trueskill.Rating(mu=25.000, sigma=8.333),
 'beachside belle ends': trueskill.Rating(mu=25.000, sigma=8.333),
 'chick park': trueskill.Rating(mu=25.000, sigma=8.333),
 'como cunstables': trueskill.Rating(mu=25.000, sigma=8.333),
 'coolbinia cruisers': trueskill.Rating(mu=25.000, sigma=8.333),
 'dianella dirty street pies': trueskill.Rating(mu=25.000, sigma=8.333),
 'dirty herdy birdies': trueskill.Rating(mu=25.000, sigma=8.333),
 'fremantle cappuccino strippers': trueskill.Rating(mu=25.000, sigma=8.333),
 'freopian tubes': trueskill.Rating(mu=25.000, sigma=8.333),
 'freovaries': trueskill.Rating(mu=25.000, sigma=8.333),
 'jolimont flying vs': trueskill.Rating(mu=25.000, sigma=8.333),
 'joondnanna jerryhatricks': trueskill.Rating(mu=25.000, sigma=8.333),
 'perth on heat': trueskill.Rating(mu=25.000, sigma=8.333)}

In [17]:
# Read in the old ratings from csv, make sure they're init'd as Rating() objects
ratings_past = pd.read_csv(oldskill_csv)
for i in range(0,len(ratings_past)):
               ratings_past['Rating'][i] = Rating(mu=ratings_past['mu'][i],)
                                                  #sigma=ratings_past['sigma'][i]#)
                                                    # This is commented out because we want to try starting from default sigma values
                                                    # Because new teams enter the mix, we are not as certain of each team's skill
ratings_past.head()

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Unnamed: 0,Team,Rating,mu,sigma
0,0,applecross dressers,"trueskill.Rating(mu=21.785, sigma=8.333)",21.784672,2.955955
1,1,como cunstables,"trueskill.Rating(mu=29.029, sigma=8.333)",29.028743,3.063469
2,2,fremantle cappuccino strippers,"trueskill.Rating(mu=26.050, sigma=8.333)",26.050444,2.832910
3,3,freopian tubes,"trueskill.Rating(mu=32.085, sigma=8.333)",32.084832,2.831814
4,4,jolimont flying vs,"trueskill.Rating(mu=25.420, sigma=8.333)",25.420201,2.833605


In [18]:
# Get the old ratings (last season), send them to a new dictionary
oldratings=ratings_past.set_index('Team')['Rating'].to_dict()
oldratings;

In [19]:
# Get the old ratings and the ratings for the new teams
# Merge the two
mergedratings = skill.copy()
mergedratings.update(oldratings)
mergedratings

{'applecross dressers': trueskill.Rating(mu=21.785, sigma=8.333),
 'ardross a-rerollers': trueskill.Rating(mu=25.000, sigma=8.333),
 'bayswatermelon breezers': trueskill.Rating(mu=25.000, sigma=8.333),
 'beachside belle ends': trueskill.Rating(mu=25.000, sigma=8.333),
 'beachside bellends': trueskill.Rating(mu=32.798, sigma=8.333),
 'chick park': trueskill.Rating(mu=30.466, sigma=8.333),
 'como cunstables': trueskill.Rating(mu=29.029, sigma=8.333),
 'coolbinia cruisers': trueskill.Rating(mu=16.244, sigma=8.333),
 'dianella dirty street pies': trueskill.Rating(mu=9.484, sigma=8.333),
 'dirty herdy birdies': trueskill.Rating(mu=25.000, sigma=8.333),
 'fremantle cappuccino strippers': trueskill.Rating(mu=26.050, sigma=8.333),
 'freopian tubes': trueskill.Rating(mu=32.085, sigma=8.333),
 'freovaries': trueskill.Rating(mu=18.209, sigma=8.333),
 'jolimont flying vs': trueskill.Rating(mu=25.420, sigma=8.333),
 'joondnanna jerryhatricks': trueskill.Rating(mu=27.852, sigma=8.333),
 'nedlands yu

In [20]:
# Update the ratings
# Go through each row in winloss results, update the ratings of both teams
# Rating(TeamWhoWon, TeamWhoLost, DidTheGameDraw)
for i in range(0,len(winloss)):
    mergedratings[winloss['Winteam'][i]],mergedratings[winloss['Lossteam'][i]] = rate_1vs1(mergedratings[winloss['Winteam'][i]],
                                                                                      mergedratings[winloss['Lossteam'][i]],
                                                                                      drawn=winloss['IsDraw'][i])

mergedratings

{'applecross dressers': trueskill.Rating(mu=20.873, sigma=3.298),
 'ardross a-rerollers': trueskill.Rating(mu=26.882, sigma=2.786),
 'bayswatermelon breezers': trueskill.Rating(mu=16.727, sigma=3.299),
 'beachside belle ends': trueskill.Rating(mu=26.839, sigma=2.991),
 'beachside bellends': trueskill.Rating(mu=32.798, sigma=8.333),
 'chick park': trueskill.Rating(mu=37.260, sigma=3.511),
 'como cunstables': trueskill.Rating(mu=29.613, sigma=3.348),
 'coolbinia cruisers': trueskill.Rating(mu=27.736, sigma=3.011),
 'dianella dirty street pies': trueskill.Rating(mu=17.740, sigma=3.440),
 'dirty herdy birdies': trueskill.Rating(mu=31.676, sigma=2.946),
 'fremantle cappuccino strippers': trueskill.Rating(mu=27.388, sigma=3.181),
 'freopian tubes': trueskill.Rating(mu=29.393, sigma=2.742),
 'freovaries': trueskill.Rating(mu=18.832, sigma=3.443),
 'jolimont flying vs': trueskill.Rating(mu=20.735, sigma=3.343),
 'joondnanna jerryhatricks': trueskill.Rating(mu=25.782, sigma=3.020),
 'nedlands y

In [21]:
# Convert the new ratings from a dict to a dataframe
# Make sure to extract mu and sigma values for each team for ease of importing the rating objects
# Storing as a csv will turn the rating objects back in to text

skill_df = DataFrame(mergedratings.items(),columns=['Team','Rating'])

skill_df['mu'] = None
skill_df['sigma'] = None
for i in range(0,len(skill_df)):
    skill_df['mu'][i]=skill_df['Rating'][i].mu
    skill_df['sigma'][i]=skill_df['Rating'][i].sigma
    #skill_df['Team'][i] = skill_df['Team'][i].lower()

# Write the newskill results to wherever was specified
skill_df.to_csv(newskill_csv)
skill_df.head()

,Team,Rating,mu,sigma
0,como cunstables,"trueskill.Rating(mu=29.613, sigma=3.348)",29.6131,3.34799
1,beachside belle ends,"trueskill.Rating(mu=26.839, sigma=2.991)",26.8387,2.99118
2,joondnanna jerryhatricks,"trueskill.Rating(mu=25.782, sigma=3.020)",25.7816,3.01983
3,jolimont flying vs,"trueskill.Rating(mu=20.735, sigma=3.343)",20.7353,3.34288
4,bayswatermelon breezers,"trueskill.Rating(mu=16.727, sigma=3.299)",16.7266,3.29853


In [102]:
# Use this to re init the ratings from text back in to rating objects
# Doesn't necessarily need to run

skill_df = pd.read_csv(newskill_csv)
for i in range(0,len(skill_df)):
               skill_df['Rating'][i] = Rating(mu=skill_df['mu'][i],
                                                sigma=skill_df['sigma'][i])
skill_df.head()

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Unnamed: 0,Team,Rating,mu,sigma
0,0,mosman park milkmen,"trueskill.Rating(mu=25.650, sigma=8.333)",25.650328,8.333333
1,1,south perth major blazers,"trueskill.Rating(mu=19.111, sigma=2.775)",19.111039,2.775426
2,2,gmla jurassic pucks,"trueskill.Rating(mu=19.049, sigma=3.090)",19.048920,3.090409
3,3,creepy crawleys,"trueskill.Rating(mu=23.973, sigma=8.333)",23.972845,8.333333
4,4,mt pleasant mounties,"trueskill.Rating(mu=30.027, sigma=3.203)",30.026670,3.203453


In [106]:
import winsound
winsound.Beep(2000,500)